In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
import pandas as pd
import warnings
import numpy as np
import os
warnings.filterwarnings('ignore')
train = pd.read_csv('/gdrive/My Drive/Data/trains.csv')
test = pd.read_csv('/gdrive/My Drive/Data/test.csv')

for col in train.columns:
    col_type = train[col].dtypes
    min1 = train[col].min()
    max1 = train[col].max()
    if str(col_type)[:3] == 'int':
        train[col] = train[col].astype(np.int16)
    else:
        if min1 > np.finfo(np.float16).min and max1 < np.finfo(np.float16).max:
            train[col] = train[col].astype(np.float16)
        elif min1 > np.finfo(np.float32).min and max1 < np.finfo(np.float32).max:
            train[col] = train[col].astype(np.float32)
        else:
            train[col] = train[col].astype(np.float64)
train = train.sample(frac=1,random_state=123).reset_index(drop=True)
train_X = train.iloc[:,4:]
train_Y = train.iloc[:,0:4]
test_X = test.iloc[:,1:]

In [3]:
import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.utils import get_custom_objects
from keras.layers import *
from keras import optimizers
from keras import activations


def res_unit(inputs, channels):
    x = BatchNormalization()(inputs)
    x = Activation('mish')(x)
    x = Conv1D(channels, kernel_size=3, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('mish')(x)
    x = Conv1D(channels, kernel_size=3, padding='same', use_bias=False)(x)
    added = Add()([inputs, x])
    return added

def res_unit_stride(inputs, channels):
    x = BatchNormalization()(inputs)
    x = Activation('mish')(x)
    x = Conv1D(channels, kernel_size=3, strides=2, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('mish')(x)
    x = Conv1D(channels, kernel_size=3, padding='same', use_bias=False)(x)
    conv = Conv1D(channels, kernel_size=1, strides=2, padding='same', use_bias=False)(inputs)
    added = Add()([conv, x])
    return added

class mish(Activation):
    def __init__(self, activation, **kwargs):
        super(mish, self).__init__(activation, **kwargs)
        self.__name__ = 'mish'

def Mish(x):
    return x*K.tanh(K.softplus(x))

get_custom_objects().update({'mish': mish(Mish)}) 

Using TensorFlow backend.


In [0]:
seq_in = Input(shape=(226,1))
x = Conv1D(16, kernel_size=3, activation='relu', padding='valid')(seq_in)
x = MaxPooling1D(2, padding='valid')(x)
x =  res_unit(x, 16)
x =  res_unit(x, 16)
x =  res_unit(x, 16)
x =  res_unit_stride(x, 32)
x =  res_unit(x, 32)
x =  res_unit(x, 32)
x =  res_unit(x, 32)
x =  res_unit_stride(x, 64)
x =  res_unit(x, 64)
x =  res_unit(x, 64)
x =  res_unit(x, 64)
x =  res_unit(x, 64)
x =  res_unit_stride(x, 128)
x =  res_unit(x, 128)
x =  res_unit(x, 128)
x =  res_unit(x, 128)
x =  res_unit_stride(x, 256)
x =  res_unit(x, 256)
x =  res_unit(x, 256)
x = BatchNormalization()(x)
x = Activation('mish')(x)
x = GlobalAveragePooling1D()(x)
seq_out = Dense(226, kernel_initializer='he_normal', activation='linear')(x)
seq_out = Reshape((226,1))(seq_out)

model = Model(inputs=seq_in , outputs=[seq_out,x])

In [0]:
op = optimizers.Adam(lr = 0.001 , beta_1 = 0.9, beta_2 = 0.999)
model.compile(loss='mae', optimizer=op, metrics=['mae'])

In [15]:
from keras.callbacks import Callback
import matplotlib.pyplot as plt
import keras.backend as K
plt.figure(figsize = (13,8))
class LRFinder(Callback):
    
    def __init__(self, min_lr=1e-5, max_lr=1e-2, steps_per_epoch=None, epochs=None):
        super().__init__()
        self.total_iterations = steps_per_epoch * epochs
        self.min_lr = min_lr
        self.max_lr = max_lr
        self.iteration = 0
        self.history = {}
        
    def clr(self):
        '''Calculate the learning rate.'''
        x = self.iteration / self.total_iterations 
        return self.min_lr + (self.max_lr-self.min_lr) * x
        
    def on_train_begin(self, logs=None):
        '''Initialize the learning rate to the minimum value at the start of training.'''
        logs = logs or {}
        K.set_value(self.model.optimizer.lr, self.min_lr)
        
    def on_batch_end(self, epoch, logs=None):
        '''Record previous batch statistics and update the learning rate.'''
        logs = logs or {}
        self.iteration += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.iteration)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
            
        K.set_value(self.model.optimizer.lr, self.clr())
    def plot_lr(self):
        '''Helper function to quickly inspect the learning rate schedule.'''
        plt.plot(self.history['iterations'], self.history['lr'])
        plt.yscale('log')
        plt.xlabel('Iteration')
        plt.ylabel('Learning rate')
        
    def plot_loss(self):
        '''Helper function to quickly observe the learning rate experiment results.'''
        plt.plot(self.history['lr'], self.history['loss'])
        plt.xscale('log')
        plt.xlabel('Learning rate')
        plt.ylabel('Loss')

<Figure size 936x576 with 0 Axes>

In [0]:
epochs = 100
batch_size = 1000
epoch_size = len(train_X)

lr_finder = LRFinder(min_lr=1e-5, 
                     max_lr=1e-3, 
                     steps_per_epoch=np.ceil(epoch_size/batch_size), 
                     epochs=epochs)

early_stop = keras.callbacks.EarlyStopping(patience=20, monitor='val_loss')

In [0]:
model.load_weights('/gdrive/My Drive/ckpt2/encoder_01_valloss0.00.hdf5')

In [0]:
x1 = model.outputs[1]
x1 = Dense(804, kernel_initializer='he_normal',activation='mish')(x1)
x1 = Dense(644,activation='mish')(x1)
x1 = BatchNormalization()(x1)
x1 = Dense(484, activation='mish')(x1)
x1 = Dense(324,activation='mish')(x1)
x1 = BatchNormalization()(x1)
x1 = Dense(164,activation='mish')(x1)
outputs = Dense(4,activation='linear')(x1)

new_model = Model(inputs=model.inputs, outputs=outputs)

In [29]:
new_model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 226, 1)       0                                            
__________________________________________________________________________________________________
conv1d_130 (Conv1D)             (None, 224, 16)      64          input_4[0][0]                    
__________________________________________________________________________________________________
max_pooling1d_4 (MaxPooling1D)  (None, 112, 16)      0           conv1d_130[0][0]                 
__________________________________________________________________________________________________
batch_normalization_120 (BatchN (None, 112, 16)      64          max_pooling1d_4[0][0]            
____________________________________________________________________________________________

In [0]:
new_model.compile(loss='mae', optimizer=op,metrics=['mae'])
train_X_se = np.expand_dims(train_X, axis=2)
ckpt_dir = '/gdrive/My Drive/ckpt3'
ckpt_path = ckpt_dir + '/modeling_{epoch:02d}_valloss{val_loss:.2f}.hdf5'
ckpt = keras.callbacks.ModelCheckpoint(ckpt_path, monitor='val_loss', verbose=0, save_best_only=True, mode='min')

In [39]:
new_model.fit(train_X_se, train_Y, epochs=100, batch_size=1024, callbacks=[ckpt,lr_finder], validation_split=0.15,shuffle=True)

Train on 688500 samples, validate on 121500 samples
Epoch 1/100
688500/688500 [==============================] - 129s 188us/step - loss: 0.9490 - mean_absolute_error: 0.9490 - val_loss: 0.6724 - val_mean_absolute_error: 0.6724
Epoch 2/100
135168/688500 [====>.........................] - ETA: 1:37 - loss: 0.9375 - mean_absolute_error: 0.9375

KeyboardInterrupt: ignored